In [2]:
import ee
import os
import geojson
#import matplotlib.pyplot as plt
#import numpy as np
import IPython.display as disp
#from geemap import png_to_gif
#from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
import requests
import time
ee.Initialize()
%matplotlib inline

In [3]:
os.listdir("Locations/")

['algeciras.geojson',
 'ferrol.geojson',
 'gibraltar.geojson',
 'gibraltar_peq.geojson',
 'map.geojson',
 'shangai.geojson',
 'south_sicilia.geojson']

In [20]:
filename = "gibraltar.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


start_date = "2017-12-01" # 
end_date = "2021-12-31" #

ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

#import time
#acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
#[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

im_list = ffa_db.toList(ffa_db.size())
index = 9

mode = "VH"

if mode == "VV":
    min_vision = -30
    max_vision = 0
elif mode == "VH":
    min_vision = -20
    max_vision = 0

url = ee.Image(im_list.get(index)).select(mode).clip(aoi).getThumbURL({'min': min_vision, 'max': max_vision})
disp.Image(url=url, width=1200)

In [21]:
acq_times = ffa_db.aggregate_array('system:time_start').getInfo()

len([time.strftime('%T %d/%m/%y', time.gmtime(acq_time/1000)) for acq_time in acq_times])

1088

In [12]:
time.gmtime(acq_times[0]/1000)

time.struct_time(tm_year=2021, tm_mon=12, tm_mday=1, tm_hour=18, tm_min=26, tm_sec=23, tm_wday=2, tm_yday=335, tm_isdst=0)

In [ ]:
ee.Image(im_list.get(index)).select('VH').clip(aoi).getInfo()

In [ ]:

img_data = requests.get(url).content
with open('Images/prueba2.png', 'wb') as handler:
    handler.write(img_data)